In [ ]:
# pacotes

import requests
import pandas as pd
import json as json
import psycopg2
from sqlalchemy import create_engine, MetaData
from sqlalchemy import MetaData, insert, text
from sqlalchemy.dialects.postgresql import insert
from urllib.parse import quote_plus
from dotenv import load_dotenv
import os
import re
import unicodedata


In [ ]:
# Caminho para o .env 
dotenv_path = "URL"

# Carrega as variáveis do arquivo .env, forçando substituição se já houver algo na memória
load_dotenv(dotenv_path, override=True)

# Recupera variáveis de ambiente
host = os.getenv("host")
porta = os.getenv("porta")
usuario = os.getenv("usuario")
senha = quote_plus(os.getenv("senha"))  # Protege caracteres especiais
banco = os.getenv("database")

# Mostra para confirmação
print("Conectando em:", host, porta, usuario, banco)

# Cria a engine
engine = create_engine(f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco}")

In [ ]:
# Carrega o arquivo com as credenciais

with open('CREDENCIAIS') as f:
    config = json.load(f)

token = config['TOKEN_API']

In [ ]:
# API PEDIDOS

pagina_pedidos = 1
todos_dados_pedidos = []
limite_paginas = 1000000  

while True:
    url_pedidos = f"API/PEDIDOS"

    headers_pedidos = {
        "Content-Type": "application/json",
        "Token": token
    }

    response = requests.get(url_pedidos, headers=headers_pedidos)
    
    if response.status_code == 200:
        dados_pedidos = response.json()
        
        if not dados_pedidos:
            break

        todos_dados_pedidos.extend(dados_pedidos)
        print(f"✅ Página {pagina_pedidos} carregada. Total acumulado: {len(todos_dados_pedidos)}")

        pagina_pedidos += 1

        # Interrompe
        if pagina_pedidos > limite_paginas:
            print("🚧 Limite de páginas atingido (teste)")
            break

    else:
        print(f"❌ Erro na página {pagina_pedidos}: {response.status_code} - {response.text}")
        break

# Converte em DataFrame
df_pedidos = pd.DataFrame(todos_dados_pedidos)
print(df_pedidos.head())

In [ ]:
# API OP

pagina = 1
todos_dados = []

# Loop de paginação
while True:
    url = f"API/OP"

    headers = {
        "Content-Type": "application/json",
        "Token": token
    }

    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        dados = response.json()
        
        # Se a resposta vier vazia, terminamos a coleta
        if not dados:
            break

        todos_dados.extend(dados)
        print(f"✅ Página {pagina} carregada. Total acumulado: {len(todos_dados)}")
        pagina += 1
    else:
        print(f"❌ Erro na página {pagina}: {response.status_code} - {response.text}")
        break

# Converter em DataFrame
df_ops = pd.DataFrame(todos_dados)

# Visualizar
print("🔍 Primeiras ordens de produção:")
print(df_ops.head())

In [ ]:
df_ops.head(2)

In [ ]:
# converte a coluna dt_inicio para datetime
df_ops['dt_inicio'] = pd.to_datetime(df_ops['dt_inicio'], format="%d/%m/%Y", errors="coerce")

# agora ordena pelos mais recentes
df_ops.sort_values(by="dt_inicio", ascending=False).head()

In [ ]:
# Quebrando o array dos ITENS 

# Explodir para transformar listas de dicionários em múltiplas linhas
df_explodido = df_ops.explode('itens').reset_index(drop=True)

# Normalizar os dicionários (cada chave vira uma coluna)
df_itens = pd.json_normalize(df_explodido['itens'])

# Concatenar com os dados da OP (sem a coluna original 'itens')
df_op_itens = pd.concat([df_explodido.drop(columns=['itens']), df_itens], axis=1)

In [ ]:
# Funcao para evitar caracteres indesejados

def normalizar_texto(texto):
    if not isinstance(texto, str):
        return ""
    texto = texto.strip().upper()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = re.sub(r'[^A-Z0-9]', '', texto)  # mantém apenas letras e números
    return texto

def limpar_coluna_texto(df, nome_coluna):
    df[nome_coluna] = df[nome_coluna].apply(normalizar_texto)
    return df

In [ ]:
df_op_itens.head()

In [ ]:
# Adicionando a coluna de quantidade total 

df_op_itens ['qtd_total'] = df_op_itens ['qtde'] + df_op_itens ['qtde_b']

In [ ]:
# Definindo as colunas necessarias

df_op_itens = df_op_itens [['numero','codigo','dt_inicio','pedido','codcli','cor','qtde','qtde_b','qtd_total']]

In [ ]:
df_op_itens.head()

In [ ]:
# Cria uma segunda coluna com o codigo do produto e uma segunda coluna com a cor e depois cria a chave na df op

df_op_itens ['codigo2'] = df_op_itens ['codigo']
df_op_itens ['cor2'] = df_op_itens ['cor']

# aplica a funcao
df_op_itens = limpar_coluna_texto(df_op_itens, 'codigo2')
df_op_itens = limpar_coluna_texto(df_op_itens, 'cor2')

# cria a chave
df_op_itens ['chave_produto_cor'] = df_op_itens ['codigo2'] + "-" + df_op_itens ['cor2']

In [ ]:
df_op_itens.head()

In [ ]:
# selecionando as colunas necessarias

df_op_itens = df_op_itens [['numero','codigo','dt_inicio','pedido','codcli','cor','qtde','qtde_b','qtd_total','chave_produto_cor']]

In [ ]:
# rename

df_op_itens.rename(columns={

    'numero':'codigoOp',
    'codcli':'codigo_cliente', # preciso alterar no banco
    'codigo':'codigo_produto', # preciso alterar no banco
    'cor':'codigoCor',
    'qtd_total':'qtdProduzir',
    'dt_inicio':'dataCriacaoOp'
    
},inplace=True)

In [ ]:
# verificando

df_op_itens.head()

In [ ]:
# verificando

df_pedidos [['numero','entrega']].head(2)

Basicamente, vamos fazer um merge dos dados de PEDIDOS para OP. O Objetivo é trazer a data da entrega (prevista) para o dataframe de Ordens de Produção, pra não precisar depender de relacionamentos futuros.

In [ ]:
# Verificando quantos registros conseguimos preencher (que dão match nas duas tabelas) e quantos registros ficarão com data prevista null.

lista_pedidos = df_pedidos['numero'].drop_duplicates().tolist()

# copia do df
df_op_itens_v2 = df_op_itens

# flag
df_op_itens_v2['flag_validacao'] = df_op_itens_v2['pedido'].isin(lista_pedidos).astype(int)
df_op_itens_v2['flag_validacao'].value_counts()

In [ ]:
# Contabilizando os pedidos

qtd_pedidos_validacao = df_op_itens_v2.query("flag_validacao == 0").nunique()
qtd_pedidos_validacao

Basicamente os registros com data prevista null ficam em 1% do total da base. Por isso, optamos por adicionar uma data genérica para esses casos pontuais

In [ ]:
# realizando o merge

df_op_itens = pd.merge(df_op_itens,df_pedidos[['numero','entrega']],how='left',left_on='pedido',right_on='numero')
df_op_itens.drop(columns=['numero'],inplace=True)
df_op_itens.head()

In [ ]:
# trocando o nome da data

df_op_itens.rename(columns={'entrega':'dataPrevistaEntrega'},inplace=True)
df_op_itens.columns

In [ ]:
# quantos dados ficaram null ?

df_op_itens.isnull().sum()

In [ ]:
# copiando o df principal
df_op_itens_validacao = df_op_itens.copy()

# Lista de pedidos válidos
lista_pedidos = df_pedidos['numero'].astype(str).tolist() 

# Garantir que a coluna 'pedido' também seja string para evitar erros de comparação
df_op_itens_validacao['pedido'] = df_op_itens_validacao['pedido'].astype(str)

#  Criar flag de validação: 1 se existe na lista de pedidos, 0 caso contrário
df_op_itens_validacao['validacao'] = df_op_itens_validacao['pedido'].isin(lista_pedidos).astype(int)

# Filtrar os que NÃO existem na base de pedidos
df_op_itens_nao_encontrados = df_op_itens_validacao.query("validacao == 0")


In [ ]:
df_op_itens_nao_encontrados.info()

In [ ]:
# Lista de pedidos 

df_op_itens_teste = df_op_itens
df_pedidos_lista = df_pedidos['numero'].tolist()
df_op_itens_teste ['validacao'] = df_op_itens_teste['pedido'].isin(df_pedidos_lista).astype(int)
df_op_itens_teste ['validacao'].value_counts()

In [ ]:
# df somente com os nao encontrados

df_pedidos_nao_encontrados = df_op_itens_teste.query("validacao == 0")
qtd_pedidos_nao_encontrados = df_pedidos_nao_encontrados['pedido'].nunique()
qtd_pedidos_nao_encontrados

In [ ]:
# criando uma função pra preencher valores null da coluna entrega

def preenchedata(df, coluna, datanova):
    df[coluna] = df[coluna].fillna(datanova)
    return df

df_op_itens = preenchedata(df_op_itens, 'dataPrevistaEntrega', '01/01/1900')

In [ ]:
df_op_itens.info()

In [ ]:
df_op_itens.head()

In [ ]:
# transformando a coluna para date

df_op_itens['dataCriacaoOp'] = pd.to_datetime(df_op_itens['dataCriacaoOp'], dayfirst=True, errors="coerce")
df_op_itens['dataPrevistaEntrega'] = pd.to_datetime(df_op_itens['dataPrevistaEntrega'], dayfirst=True, errors="coerce")

In [ ]:
df_op_itens.head()

In [ ]:
df_op_itens.sort_values(by='dataCriacaoOp', ascending=False).head()

In [ ]:
df_op_itens.drop(columns=['flag_validacao','validacao'],inplace=True)

In [ ]:
qtdteste1 = df_op_itens ['codigoOp'].nunique()
qtdteste2 = df_op_itens ['codigoOp'].count()

print(qtdteste1)
print(qtdteste2)

In [ ]:
df_op_itens_testedate = df_op_itens.query("dataCriacaoOp == '2025-08-18'")
df_op_itens_testedate.head()

In [ ]:
# salvando 

metadata = MetaData()
metadata.reflect(bind=engine)
tabela_op = metadata.tables["FatoOrdemProducao"]

# garanta tipo e zeros à esquerda se for o caso
df_op_itens["codigoOp"] = df_op_itens["codigoOp"].astype(str)

# remove duplicadas no DF
df_op_itens = df_op_itens.drop_duplicates(subset=["codigoOp"], keep="last")

# troca NaT/NaN por None (NULL)
# df_op_itens = df_op_itens.replace({pd.NaT: None, np.nan: None}).where(pd.notnull(df_op_itens), None)

dados = df_op_itens.to_dict(orient="records")

stmt = insert(tabela_op).values(dados)
stmt = stmt.on_conflict_do_nothing(index_elements=["codigoOp"])

with engine.begin() as conn:
    conn.execute(stmt)